In [1]:
# Mounting Google Drive to access files in Colab
from google.colab import drive
drive.mount('/content/drive')

# Importing the libraries we need for our project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import files
from textblob import TextBlob
import spacy
from collections import Counter
from wordcloud import WordCloud
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install the OpenAI Python client at version 0.28
!pip install openai==0.28

# Importing the libraries we need for working with OpenAI and transformers
import openai
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Set your OpenAI API key (make sure to replace with your actual API key)
os.environ["OPENAI_API_KEY"] = "your_api_key_by_openAI"

# Make sure the key is available for OpenAI functions
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# Load the spaCy model for keyword extraction
nlp = spacy.load('en_core_web_sm')  # This loads a small English model for processing text

In [4]:
# Creating the basic structure as a Python script that can later be converted into a Jupyter Notebook

# Introduction: Load necessary libraries
def introduction():
    print("This notebook is designed to evaluate candidates' introduction videos using emotion scores, transcript scores, and transcript text.")

# Load & Preprocess Data
def load_and_preprocess_data():
    """
    Loads and preprocesses data from specified directories in Google Drive.
    It retrieves emotion data, transcript scores, and transcript text for analysis.
    Returns three lists containing the loaded data.
    """
    emotion_data_dir = "/content/drive/MyDrive/I'mBesideYou/emotion_data"
    transcript_data_dir = "/content/drive/MyDrive/I'mBesideYou/transcript_data"
    transcript_text_dir = "/content/drive/MyDrive/I'mBesideYou/transcripts"

    emotion_data = []
    transcript_data = []
    transcript_text = []

    for folder in os.listdir(emotion_data_dir):
        emotion_csv_path = os.path.join(emotion_data_dir, folder, 'emotion.csv')
        gaze_csv_path = os.path.join(emotion_data_dir, folder, 'gaze.csv')
        metadata_csv_path = os.path.join(emotion_data_dir, folder, 'metadata.csv')

        if os.path.exists(emotion_csv_path) and os.path.exists(gaze_csv_path) and os.path.exists(metadata_csv_path):
            emotion_csv = pd.read_csv(emotion_csv_path)
            gaze_csv = pd.read_csv(gaze_csv_path)
            metadata_csv = pd.read_csv(metadata_csv_path)
            emotion_data.append({'emotion': emotion_csv, 'gaze': gaze_csv, 'metadata': metadata_csv})

    for file in os.listdir(transcript_data_dir):
        if file.endswith('.csv'):
            transcript_scores = pd.read_csv(os.path.join(transcript_data_dir, file))
            transcript_data.append(transcript_scores)

    for file in os.listdir(transcript_text_dir):
        if file.endswith('.txt'):
            with open(os.path.join(transcript_text_dir, file), 'r') as f:
                transcript_text.append(f.read())

    return emotion_data, transcript_data, transcript_text


In [5]:
# Perform EDA on Emotion Scores
def eda_emotion_data(emotion_data, plot_count, show_plots=False):
    """
    Conducts exploratory data analysis on emotion scores for each candidate.
    Generates line plots of emotion data and saves them as PNG files if requested.
    """
    for idx, candidate in enumerate(emotion_data, start=plot_count):
        emotion_df = candidate['emotion']

        if show_plots:
            plt.figure(figsize=(10, 6))
            sns.lineplot(data=emotion_df)
            plt.title(f"Emotion Data Overview for Candidate {idx}")
            plt.xlabel("Time")
            plt.ylabel("Emotion Score")
            plt.savefig(f'emotion_plot_{idx}.png')
            plt.close()
            plt.show()

# Perform EDA on Transcript Data
def eda_transcript_data(transcript_data, plot_count, show_plots=False):
    """
    Performs exploratory data analysis on transcript data, including speech speed and emotion scores.
    Generates and saves histograms and box plots for various attributes if requested.
    """
    for plot_count, transcript in enumerate(transcript_data, start=1):
        if show_plots:
            plt.figure(figsize=(8, 4))
            sns.histplot(transcript['speech_speed'], kde=True, bins=20)
            plt.title(f"Speech Speed Distribution - Candidate {plot_count}")
            plt.xlabel("Speech Speed")
            plt.ylabel("Frequency")
            plt.savefig(f'speech_speed_plot_{plot_count}.png')
            plt.show()

            plt.figure(figsize=(10, 6))
            sns.boxplot(data=transcript[['positive', 'negative', 'neutral']])
            plt.title(f"Emotion Scores Distribution - Candidate {plot_count}")
            plt.xlabel("Emotion Type")
            plt.ylabel("Score")
            plt.savefig(f'emotion_scores_plot_{plot_count}.png')
            plt.show()

            plt.figure(figsize=(8, 4))
            sns.histplot(transcript['confident'], kde=True, color='green', label='Confident')
            sns.histplot(transcript['hesitant'], kde=True, color='red', label='Hesitant')
            plt.title(f"Confident vs Hesitant Distribution - Candidate {plot_count}")
            plt.xlabel("Score")
            plt.ylabel("Frequency")
            plt.legend()
            plt.savefig(f'confident_hesitant_plot_{plot_count}.png')
            plt.show()

            plt.figure(figsize=(8, 4))
            sns.histplot(transcript['concise'], kde=True, color='blue', label='Concise')
            sns.histplot(transcript['enthusiastic'], kde=True, color='orange', label='Enthusiastic')
            plt.title(f"Conciseness vs Enthusiasm Distribution - Candidate {plot_count}")
            plt.xlabel("Score")
            plt.ylabel("Frequency")
            plt.legend()
            plt.savefig(f'concise_enthusiastic_plot_{plot_count}.png')
            plt.show()

In [6]:
# Analyzing the transcript text
def eda_transcript_text(transcript_text, show_plots=False):
    """
    Generates and displays a word cloud for each transcript text.
    Displays the word cloud if requested.
    """
    for text in transcript_text:
        if show_plots:
            wordcloud = WordCloud(width=800, height=400).generate(text)
            plt.figure(figsize=(10, 5))
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis('off')
            plt.show()

# Sentiment Analysis
def analyze_sentiment(transcript_text):
    """
    Analyzes sentiment polarity for each transcript text using TextBlob.
    Returns a list of sentiment scores ranging from -1 (negative) to 1 (positive).
    """
    sentiment_scores = []
    for text in transcript_text:
        analysis = TextBlob(text)
        sentiment_scores.append(analysis.sentiment.polarity)
    return sentiment_scores

# Keyword Extraction
def extract_keywords(transcript_text):
    """
    Extracts and counts the most common keywords from the transcript text.
    Returns a list of the top 10 keywords after removing stopwords and non-alphabetic tokens.
    """
    keywords = []
    for text in transcript_text:
        doc = nlp(text)
        keywords.extend([token.lemma_ for token in doc if token.is_alpha and not token.is_stop])
    return Counter(keywords).most_common(10)

# Clustering Candidates
def cluster_candidates(transcript_data, n_clusters):
    """
    Clusters candidates based on average speech speed and emotion scores.
    Returns an array of cluster labels corresponding to each candidate.
    """
    features = []
    for transcript in transcript_data:
        features.append([transcript['speech_speed'].mean(), transcript['confident'].mean(), transcript['hesitant'].mean()])

    kmeans = KMeans(n_clusters=n_clusters)
    clusters = kmeans.fit_predict(features)
    return clusters

# Conclusion
def conclusion():
    """
    Prints a concluding statement summarizing the analysis process.
    Indicates that decisions can be made based on the analysis.
    """
    print("This concludes the analysis. Based on the EDA and prompt engineering, recruitment decisions can be made.")

# Download all saved plots
def download_all_plots(plot_count):
    """
    Downloads all saved plot images based on the provided plot count.
    Notifies if a file does not exist for any given plot.
    """
    for i in range(1, plot_count + 1):
        filenames = [
            f'emotion_plot_{i}.png',
            f'speech_speed_plot_{i}.png',
            f'emotion_scores_plot_{i}.png',
            f'confident_hesitant_plot_{i}.png',
            f'concise_enthusiastic_plot_{i}.png'
        ]
        for filename in filenames:
            if os.path.isfile(filename):
                files.download(filename)
            else:
                print(f"File {filename} does not exist.")

In [7]:
# Summarize actionable insights

# Function to analyze transcript text using a specified model
def analyze_transcript(transcript_text, model="gpt-2"):
    """
    Analyzes the provided transcript text for insights using the specified model.
    Supports both GPT-2 and OpenAI models for generating responses.
    """
    if model == "gpt-2":
        from transformers import GPT2LMHeadModel, GPT2Tokenizer

        model_name = "gpt2"
        tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        model = GPT2LMHeadModel.from_pretrained(model_name)

        tokenizer.pad_token = tokenizer.eos_token

        input_text = f"Analyze the following transcript text for insights: {transcript_text}"
        inputs = tokenizer.encode_plus(input_text, return_tensors='pt', padding=True, truncation=True)

        outputs = model.generate(
            inputs['input_ids'],
            max_length=500,  # Increase this if needed
            attention_mask=inputs['attention_mask']
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response
    elif model in ["gpt-3.5-turbo", "text-davinci-003"]:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": f"Analyze the following transcript text for insights: {transcript_text}"}]
        )
        return response['choices'][0]['message']['content']
    else:
        raise ValueError("Model not supported")

# Update actionable insights function to use the new analysis function
def actionable_insights(emotion_data, transcript_data, transcript_text, model="gpt-3.5-turbo"):
    """
    Generates actionable insights for each candidate based on emotion scores,
    speech speed, and transcript analysis. Provides recruitment recommendations.
    """
    for idx, (candidate_emotion, candidate_transcript, candidate_text) in enumerate(zip(emotion_data, transcript_data, transcript_text)):
        emotion_scores = candidate_emotion['emotion']

        avg_positive = emotion_scores['happy'].mean()
        avg_negative = emotion_scores['sad'].mean()
        speech_speed = candidate_transcript['speech_speed'].mean()

        # Analyze sentiment with the specified model
        transcript_analysis = analyze_transcript(candidate_text, model)

        print(f"Candidate {idx + 1} Insights:")

        if avg_positive > avg_negative and speech_speed > 2.0:
            print("Recommendation: Recruit")
            print("Reason: Positive emotion scores and acceptable pacing.")
        else:
            print("Recommendation: Do not recruit")
            print("Reason: Low positive emotion scores or poor pacing.")

        print(f"Average Speech Speed: {speech_speed:.2f} (words per minute)")
        print(f"Transcript Analysis: {transcript_analysis}")


In [8]:
# Call functions as part of the notebook workflow
plot_count = 1
show_plots = False  # Set to False if you don't want to display plots

# Introduction
introduction()

# Load and preprocess data
emotion_data, transcript_data, transcript_text = load_and_preprocess_data()

# Perform EDA on individual datasets
eda_emotion_data(emotion_data, plot_count, show_plots)
eda_transcript_data(transcript_data, plot_count, show_plots)
eda_transcript_text(transcript_text, show_plots)

# Analyze sentiment
sentiment_scores = analyze_sentiment(transcript_text)
print(f"Sentiment Scores: {sentiment_scores}")

# Cluster candidates
n_clusters = 3  # Define the number of clusters
clusters = cluster_candidates(transcript_data, n_clusters)
print(f"Candidate Clusters: {clusters}")

# Generate actionable insights
actionable_insights(emotion_data, transcript_data, transcript_text, model="gpt-2")  # Or "gpt-3.5-turbo", "davinci", "text-davinci-003"

# Conclusion
conclusion()

# Download all saved plots
# download_all_plots(plot_count)

This notebook is designed to evaluate candidates' introduction videos using emotion scores, transcript scores, and transcript text.
Sentiment Scores: [0.16197478991596637, 0.2554226475279107, -0.09755366161616164, 0.14175347222222223, 0.07088744588744589, 0.029500891265597157, 0.33, 0.06642857142857142, 0.1441786283891547, 0.21130050505050504]
Candidate Clusters: [2 0 2 0 1 0 0 2 0 1]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depra

Candidate 1 Insights:
Recommendation: Do not recruit
Reason: Low positive emotion scores or poor pacing.
Average Speech Speed: 2.90 (words per minute)
Transcript Analysis: Analyze the following transcript text for insights:  Hi, hope you're doing well. I'm Srivats Biyani. I am a PGP finance student at IIM Co-Ecode.  Now before joining IIM Co-Ecode, what all have I done? I am a chartered accountant and  I've also cleared CFA level 1. I interned with PWC for more than three years, did my  article shift basically from there and I worked in the statutory audit department. After clearing  CA, I worked with ITC limited in the internal audit department for almost 14 months after  which I joined IIM Co-Ecode for the MBA program. Now a little bit about my skills etc. So I  am a very analytical person. I love playing with numbers, you know, data sets, apply all  sorts of statistical tools, try and understand whatever possible outcomes or interpretations  are there from those numbers. I just love

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Candidate 2 Insights:
Recommendation: Recruit
Reason: Positive emotion scores and acceptable pacing.
Average Speech Speed: 3.25 (words per minute)
Transcript Analysis: Analyze the following transcript text for insights:  My name is Michael Ramos, I am from Patna, Bihar. I have done my schooling in Commerce and then  I went up to do my graduation in B.Com Honours. I have interned as an Accounting Associate  as well as a Tax Associate wherein I got the chance to apply all the knowledge that I had  learned in my B.Com to apply them in real life scenario. Apart from Academics, I have  been involved in a lot of extracurricular activities. I was the captain of the students  committee in my final school year and was also awarded the Best Student Award at the  end of the year. When I went up to college, I was the class representative which again  is the part of the students committee. Apart from that, I have been a part of Bad Scouts  and Guide for more than 5 years where I have completed seve

...

Candidate 8 Insights:
Recommendation: Do not recruit
Reason: Low positive emotion scores or poor pacing.
Average Speech Speed: 2.82 (words per minute)
Transcript Analysis: Analyze the following transcript text for insights:  Hello, I'm Sakshi. I come from Mumbai. I did my undergraduation in mass media  with specialization in advertising. I have completed two  certification courses. One was entrepreneurship course by Turnip and  second one was foundations of management by Google. Apart from that, when I'm not  busy doing college group projects and assignments, I keep myself busy doing  various things like drawing, painting, singing, writing movie reviews on my  personal movie review account on Instagram. I also recently won an  international art competition hosted by the largest science channel on YouTube  called Krezkazad. They have over 19 million followers and it is one of my  biggest proudest achievements. Because I would like to spread  awareness about mental health issues and neur

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Candidate 9 Insights:
Recommendation: Do not recruit
Reason: Low positive emotion scores or poor pacing.
Average Speech Speed: 3.11 (words per minute)
Transcript Analysis: Analyze the following transcript text for insights:  Hello, I am Jeffrey Shepherd and I am currently pursuing postgraduate and management from  IIM Coikode. I have completed my B.Tech in Biotechnology from Heritage Institute of  Technology Kolkata, followed by my M.Tech from IIT Kharagpur. I come with an experience  of three years in the regulatory affairs domain of the pharmaceutical industry and I worked  as a medical writer in Ciro Klein Farm, Mumbai and have specialized in drug safety and risk  management. What sets me apart is the expertise I bring in with my three years of experience  and an added two years of postgraduation. Along with this, I add another dimension to  the discussion with my background in biotechnology. An added advantage is my ability to pay attention  to detail, which is well testamented in 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Candidate 10 Insights:
Recommendation: Recruit
Reason: Positive emotion scores and acceptable pacing.
Average Speech Speed: 2.58 (words per minute)
Transcript Analysis: Analyze the following transcript text for insights:  Hi, my name is Nathan Lewis. I'm a first year MBA analytics student from IIM  Kashipur. From having a consulting experience at Deloitte for three years  after graduating in engineering, I paved my way to an MBA analytics degree to  quench my thirst for exploration into an amalgamation of a world of analytics  strategy and planning. During my time in Deloitte, I worked on end-to-end  validation processes for softwares for pharmaceutical clients. Also, my ongoing  post-graduation in MBA analytics has taken me closer to the field of analytics  in business. To fuel my constant need to take responsibilities, I am a member of  the media and public relations committee of our college. It lets me be a part of  the team who plans to create and apply for a promotion and PR strat

# Candidate Evaluation Report

## Sentiment Scores
- 0.16197
- 0.25542
- -0.09755
- 0.14175
- 0.07089
- 0.02950
- 0.33
- 0.06643
- 0.14418
- 0.21130

## Candidate Clusters
- 2
- 0
- 2
- 0
- 1
- 0
- 0
- 2
- 0
- 1


## Candidate Insights

### Candidate 1 Insights
<span style="color: red;">**Recommendation:** Do not recruit</span>  
**Reason:** Low positive emotion scores or poor pacing.  
**Average Speech Speed:** 2.90 words per minute  

**Transcript Analysis:**  
> Hi, hope you're doing well. I'm Srivats Biyani. I am a PGP finance student at IIM Co-Ecode. Now before joining IIM Co-Ecode, what all have I done? I am a chartered accountant and I've also cleared CFA level 1. I interned with PWC for more than three years, ....................from those numbers. I just love the idea about getting into the ed...

### Candidate 2 Insights
<span style="color: green;">**Recommendation:** Recruit</span>  
**Reason:** Positive emotion scores and acceptable pacing.  
**Average Speech Speed:** 3.25 words per minute  

**Transcript Analysis:**  
> My name is Michael Ramos, I am from Patna, Bihar. I have done my schooling in Commerce and then I went up to do my graduation in B.Com Honours. ... I look for bigger targets so that I can test myself. I need to figure myself during the preliminary years so that when the superior targets are put, I am able to come up with the best outcomes. I am also looking for a job where I can work with the students committee to develop the skills that I have learned so far.

### Candidate 3 Insights
<span style="color: red;">**Recommendation:** Do not recruit</span>  
**Reason:** Low positive emotion scores or poor pacing.  
**Average Speech Speed:** 2.78 words per minute  

**Transcript Analysis:**  
> Hello, my name is Monique Mccormick. I'm from Guntur, Andhra Pradesh. I'm an engineering graduate in electronics and communication field. My final year project is on performance analysis of shift resistors. ... I'm a very active learner and I'm always looking for opportunities to improve my skills.

### Candidate 4 Insights
<span style="color: green;">**Recommendation:** Recruit</span>  
**Reason:** Positive emotion scores and acceptable pacing.  
**Average Speech Speed:** 3.39 words per minute  

**Transcript Analysis:**  
> My name is Michael Guzman and I am 21 years old. Hailing from a small family in Varanasi, I had been there most of my life, but MBA has given me the experience of living alone for the first time, which has been quite tough and challenging for me. ... I would love to know if I am beside you during the next year's hours. Thank you!

### Candidate 5 Insights
<span style="color: red;">**Recommendation:** Do not recruit</span>  
**Reason:** Low positive emotion scores or poor pacing.  
**Average Speech Speed:** 2.28 words per minute  

**Transcript Analysis:**  
> Hello, I am Joseph Nichols. I belong to the heritage city of Varanasi. I have done my undergraduation in earth science from Banaras Hindu University. ... This idea really resonated with me and I do wish to be a part of this awakening. Thank you so much!

### Candidate 6 Insights
<span style="color: green;">**Recommendation:** Recruit</span>  
**Reason:** Positive emotion scores and acceptable pacing.  
**Average Speech Speed:** 3.33 words per minute  

**Transcript Analysis:**  
> Hello, myself is Alexander Smith. I am a first year MBA student here at IIM Lucknow. I come from a suburban part of India. I did my B.Tech in Agriculture Engineering, my M.Tech in Food Process Engineering. ... Thank you.

### Candidate 7 Insights
<span style="color: green;">**Recommendation:** Recruit</span>  
**Reason:** Positive emotion scores and acceptable pacing.  
**Average Speech Speed:** 3.27 words per minute  

**Transcript Analysis:**  
> Hello, I am Cameron Barajas and I am thrilled to apply to this opportunity today. I recently completed my BBA in 2022. I have a flexible attitude and can lead the team while being a good team player. ... Thank you.

### Candidate 8 Insights
<span style="color: red;">**Recommendation:** Do not recruit</span>  
**Reason:** Low positive emotion scores or poor pacing.  
**Average Speech Speed:** 2.82 words per minute  

**Transcript Analysis:**  
> Hello, I'm Sakshi. I come from Mumbai. I did my undergraduation in mass media with specialization in advertising. ... That's it. Thank you.

### Candidate 9 Insights
<span style="color: red;">**Recommendation:** Do not recruit</span>  
**Reason:** Low positive emotion scores or poor pacing.  
**Average Speech Speed:** 3.11 words per minute  

**Transcript Analysis:**  
> Hello, I am Jeffrey Shepherd and I am currently pursuing postgraduate management from IIM Coikode. I have completed my B.Tech in Biotechnology from Heritage Institute of Technology Kolkata... Thank you.

### Candidate 10 Insights
<span style="color: green;">**Recommendation:** Recruit</span>  
**Reason:** Positive emotion scores and acceptable pacing.  
**Average Speech Speed:** 2.58 words per minute  

**Transcript Analysis:**  
> Hi, my name is Nathan Lewis. I'm a first-year MBA analytics student from IIM Kashipur. From having a consulting experience at Deloitte for three years after graduating in engineering... Thank you!


### Summary of Candidate Insights

#### Recruited Candidates
1. <span style="color: green;">Candidate 2: Michael Ramos</span>  
2. <span style="color: green;">Candidate 4: Michael Guzman</span>  
3. <span style="color: green;">Candidate 6: Alexander Smith</span>  
4. <span style="color: green;">Candidate 7: Cameron Barajas</span>  
5. <span style="color: green;">Candidate 10: Nathan Lewis</span>  

#### Non-Recruited Candidates
1. <span style="color: red;">Candidate 1: Srivats Biyani</span>  
2. <span style="color: red;">Candidate 3: Monique Mccormick</span>  
3. <span style="color: red;">Candidate 5: Joseph Nichols</span>  
4. <span style="color: red;">Candidate 8: Sakshi</span>  
5. <span style="color: red;">Candidate 9: Jeffrey Shepherd</span>  
